Third Party Library Imports


In [1]:
import pandas as pd
import numpy as np
import sklearn
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
import matplotlib.pyplot as plt
import seaborn as sns
import io
import glob

Loading datasets into respective dataframes

In [2]:
gagfyt_df = pd.read_csv(r'C:\Users\***\IOT defender\Gagfyt.csv')
Hide_and_seek_df = pd.read_csv(r'C:\Users\***\IOT defender\Hide_and_seek.csv')
Kenjiro_df = pd.read_csv(r'C:\Users\***\IOT defender\Kenjiro.csv')
Linux_Hajime_df = pd.read_csv(r'C:\U***\IOT defender\Linux_Hajime_df.csv')
Mirai_df = pd.read_csv(r'C:\Users\***\IOT defender\Mirai.csv')
Muhstik_df = pd.read_csv(r'C:\Users\***\IOT defender\Muhstik.csv')
Okiru_df = pd.read_csv(r'C:\Users\***\IOT defender\Okiru.csv') 

C:\Users\imade\AppData\Local\Temp\ipykernel_15160\3073905347.py:3: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  Kenjiro_df = pd.read_csv(r'C:\Users\imade\OneDrive\Bureau\IOT defender\Kenjiro.csv')


Cleannig malware sub-set dataframes

In [3]:
pdList = [gagfyt_df, Hide_and_seek_df, Kenjiro_df, Mirai_df, Muhstik_df,Okiru_df, Linux_Hajime_df]  
df = pd.concat(pdList,ignore_index=True)

oncatinating the column Traffic labeled

In [4]:
df['Traffic_Labeled'] = np.where(df['Traffic_Labeled']!="Benign", df['Traffic_Labeled'] + df['Malware'], df['Traffic_Labeled'])


df.drop(["tunnel_parents   label   detailed-label", "Tunnel", "Malware","index"], axis =1, inplace = True)

In [5]:
df.head()

,ts,uid,id.orig_h,id.orig_p,id.resp_h,id.resp_p,proto,service,duration,orig_bytes,...,conn_state,local_orig,local_resp,missed_bytes,history,orig_pkts,orig_ip_bytes,resp_pkts,resp_ip_bytes,Traffic_Labeled
0,2019-09-20 22:27:19.984081920,CjnBrQ3lWRLcSTnJV6,192.168.1.195,4512,162.248.88.215,62336,tcp,NaN,NaN,NaN,...,OTH,NaN,NaN,0,C,0,0,0,0,Malicious DDoS
1,2019-09-20 22:25:46.017224960,CHL0ex2ROJt8hD3up6,192.168.1.195,49327,162.248.88.215,62336,tcp,NaN,NaN,NaN,...,OTH,NaN,NaN,0,C,0,0,0,0,Malicious DDoS
2,2019-09-20 22:25:34.246748928,C7bfRx4kiGSypeozCd,162.248.88.215,62336,192.168.1.195,614,tcp,NaN,NaN,NaN,...,OTH,NaN,NaN,0,^c,0,0,0,0,Benign
3,2019-09-20 22:26:06.340357120,CE7K0r1ua7NTzBHGDf,192.168.1.195,29776,162.248.88.215,62336,tcp,NaN,NaN,NaN,...,OTH,NaN,NaN,0,C,0,0,0,0,Malicious DDoS
4,2019-09-20 22:23:52.564219904,CylM264HX2dYzLJAl6,162.248.88.215,62336,192.168.1.195,3389,tcp,NaN,NaN,NaN,...,OTH,NaN,NaN,0,^c,0,0,0,0,Benign


Data Pre Processing

In [6]:
df['Traffic_Labeled'] = df['Traffic_Labeled'].str.strip()

In [7]:
df['Traffic_Labeled'].value_counts()

Traffic_Labeled
Benign                                 1885954
Malicious PartOfAHorizontalPortScan    1526588
Malicious Okiru                         356530
Malicious DDoS                            2476
Malicious Attack                           199
Malicious C&C-HeartBeat                    154
Malicious C&C                                7
Name: count, dtype: int64

Converting Originating endpoint’s TCP/UDP port (or ICMP code) to Categorical

In [8]:
df["id.orig_p"]= df["id.orig_p"].values.astype(str)
df["id.resp_p"]= df["id.resp_p"].values.astype(str)

In [9]:
df

,ts,uid,id.orig_h,id.orig_p,id.resp_h,id.resp_p,proto,service,duration,orig_bytes,...,conn_state,local_orig,local_resp,missed_bytes,history,orig_pkts,orig_ip_bytes,resp_pkts,resp_ip_bytes,Traffic_Labeled
0,2019-09-20 22:27:19.984081920,CjnBrQ3lWRLcSTnJV6,192.168.1.195,4512,162.248.88.215,62336,tcp,NaN,NaN,NaN,...,OTH,NaN,NaN,0,C,0,0,0,0,Malicious DDoS
1,2019-09-20 22:25:46.017224960,CHL0ex2ROJt8hD3up6,192.168.1.195,49327,162.248.88.215,62336,tcp,NaN,NaN,NaN,...,OTH,NaN,NaN,0,C,0,0,0,0,Malicious DDoS
2,2019-09-20 22:25:34.246748928,C7bfRx4kiGSypeozCd,162.248.88.215,62336,192.168.1.195,614,tcp,NaN,NaN,NaN,...,OTH,NaN,NaN,0,^c,0,0,0,0,Benign
3,2019-09-20 22:26:06.340357120,CE7K0r1ua7NTzBHGDf,192.168.1.195,29776,162.248.88.215,62336,tcp,NaN,NaN,NaN,...,OTH,NaN,NaN,0,C,0,0,0,0,Malicious DDoS
4,2019-09-20 22:23:52.564219904,CylM264HX2dYzLJAl6,162.248.88.215,62336,192.168.1.195,3389,tcp,NaN,NaN,NaN,...,OTH,NaN,NaN,0,^c,0,0,0,0,Benign
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3771903,2018-07-25 18:25:12.036609792,Cf2JQl4Fk3in6TJJsg,192.168.100.111,29930,197.255.125.11,18246,udp,NaN,0 days 00:00:00.247373,106.0,...,SF,NaN,NaN,0,Dd,1,134,1,365,Benign
3771904,2018-07-25 22:21:24.001981952,CuUxQ5l0YFXm7aurf,192.168.100.111,864,222.176.49.129,23,tcp,NaN,0 days,0.0,...,S0,NaN,NaN,0,S,1,40,0,0,Malicious PartOfAHorizontalPortScan
3771905,2018-07-26 05:38:59.387181056,CHqHRtMd3Zx2Wt5t3,4.68.110.10,11,192.168.100.111,0,icmp,NaN,0 days,0.0,...,OTH,NaN,NaN,0,NaN,1,56,0,0,Benign
3771906,2018-07-26 07:10:22.016359936,CKjpNa3slgClcNt85,192.168.100.111,29930,59.146.147.244,52034,udp,NaN,0 days 00:00:32.010284,318.0,...,S0,NaN,NaN,0,D,3,402,0,0,Benign


Subsetting the raw data set due of Hardware Memory issue

In [10]:

df_Benign = df.loc[df['Traffic_Labeled'] == 'Benign']
df_Benign=df_Benign.loc[np.random.permutation(df_Benign.index)[:30000]]
df_Benign.reset_index(inplace=True)
df_PortScan = df.loc[df['Traffic_Labeled'] == 'Malicious PartOfAHorizontalPortScan']
df_PortScan=df_PortScan.loc[np.random.permutation(df_PortScan.index)[:20000]]
df_PortScan.reset_index(inplace=True)
df_Okiru = df.loc[df['Traffic_Labeled'] == 'Malicious Okiru']
df_Okiru = df_Okiru.loc[np.random.permutation(df_Okiru.index)[:8000]]
df_Okiru.reset_index(inplace=True)
df_Others = df[df['Traffic_Labeled'].isin(['Malicious DDoS', 'Malicious Attack', 
                                           'Malicious C&C-HeartBeat', 'Malicious C&C'])]
pdList = [df_Benign, df_Okiru, df_PortScan, df_Others]  
df = pd.concat(pdList,ignore_index=True)
df = df.drop('index',axis=1)

In [11]:
df.shape

(60836, 21)

New Variable - Classification Target

In [12]:
df.loc[df['Traffic_Labeled'] != 'Benign', 'Malicious'] = 1
df.Malicious=df.Malicious.fillna(0)
df.Malicious.value_counts()* 100 / len(df)

Malicious
1.0    50.687093
0.0    49.312907
Name: count, dtype: float64

In [13]:
df.Malicious.value_counts()* 100 / len(df)

Malicious
1.0    50.687093
0.0    49.312907
Name: count, dtype: float64

Null Values

In [14]:
df.isna().sum()* 100 / len(df)

ts                   0.000000
uid                  0.000000
id.orig_h            0.000000
id.orig_p            0.000000
id.resp_h            0.000000
id.resp_p            0.000000
proto                0.000000
service             99.516734
duration            93.518640
orig_bytes          93.518640
resp_bytes          93.518640
conn_state           0.000000
local_orig         100.000000
local_resp         100.000000
missed_bytes         0.000000
history              0.941877
orig_pkts            0.000000
orig_ip_bytes        0.000000
resp_pkts            0.000000
resp_ip_bytes        0.000000
Traffic_Labeled      0.000000
Malicious            0.000000
dtype: float64

Dropping off Variable 'uid' Unique ID of Connection, present 100% unique values

In [15]:
df.drop(['local_resp','local_orig','uid'],axis=1,inplace=True)

Transport Layer Protocol

In [16]:
df.proto.value_counts()

proto
tcp     53511
udp      6752
icmp      573
Name: count, dtype: int64

Duration: Time of last packet seen – time of first packet seen

In [17]:
from datetime import time, timedelta, datetime 
df.duration = pd.to_timedelta(df.duration)
df.duration = pd.to_timedelta(df['duration']).dt.total_seconds()

In [18]:
df[['duration','orig_bytes','resp_bytes']]=df[['duration','orig_bytes','resp_bytes']].fillna

In [19]:
df = df.fillna("Unknown")

In [20]:
df.isna().sum()

ts                 0
id.orig_h          0
id.orig_p          0
id.resp_h          0
id.resp_p          0
proto              0
service            0
duration           0
orig_bytes         0
resp_bytes         0
conn_state         0
missed_bytes       0
history            0
orig_pkts          0
orig_ip_bytes      0
resp_pkts          0
resp_ip_bytes      0
Traffic_Labeled    0
Malicious          0
dtype: int64

Variables with more than 90% unique values

In [21]:
df=df.drop(['id.resp_h','id.orig_h'],axis=1)

In [22]:
from sklearn.utils import shuffle

df.dropna(axis=0, how='any', inplace=True)

df.drop_duplicates(subset=None, keep="first", inplace=True)

df = shuffle(df)

print(df['Traffic_Labeled'].value_counts())

Traffic_Labeled
Benign                                 30000
Malicious PartOfAHorizontalPortScan    20000
Malicious Okiru                         8000
Malicious DDoS                          2476
Malicious Attack                         199
Malicious C&C-HeartBeat                  154
Malicious C&C                              7
Name: count, dtype: int64


In [23]:
df.to_csv('preprocessed_Labeled_IoT.csv', encoding='utf-8')

Feature Engineering

In [24]:
from sklearn.model_selection import train_test_split

from sklearn.preprocessing import StandardScaler

from sklearn import preprocessing

df_dumie = df

def encode_text_dummy(df_dumie, name):

    dummies = pd.get_dummies(df[name])

    for x in dummies.columns:

        dummy_name = f"{name}-{x}"

        df_dumie[dummy_name] = dummies[x]

    df_dumie.drop(name, axis=1, inplace=True)


encode_text_dummy(df_dumie,"proto")

encode_text_dummy(df_dumie,"service")

In [25]:
df_dumie.info()

<class 'pandas.core.frame.DataFrame'>
Index: 60836 entries, 12563 to 53538
Data columns (total 22 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   ts               60836 non-null  object 
 1   id.orig_p        60836 non-null  object 
 2   id.resp_p        60836 non-null  object 
 3   duration         60836 non-null  object 
 4   orig_bytes       60836 non-null  object 
 5   resp_bytes       60836 non-null  object 
 6   conn_state       60836 non-null  object 
 7   missed_bytes     60836 non-null  int64  
 8   history          60836 non-null  object 
 9   orig_pkts        60836 non-null  int64  
 10  orig_ip_bytes    60836 non-null  int64  
 11  resp_pkts        60836 non-null  int64  
 12  resp_ip_bytes    60836 non-null  int64  
 13  Traffic_Labeled  60836 non-null  object 
 14  Malicious        60836 non-null  float64
 15  proto-icmp       60836 non-null  bool   
 16  proto-tcp        60836 non-null  bool   
 17  proto-udp    

Random Forest

In [26]:
from sklearn.ensemble import RandomForestClassifier

In [27]:
import zat
from sklearn.model_selection import train_test_split
from zat.log_to_dataframe import LogToDataFrame
from zat.dataframe_to_matrix import DataFrameToMatrix
print('zat: {:s}'.format(zat.__version__))
import pandas as pd
print('Pandas: {:s}'.format(pd.__version__))
import numpy as np
print('Numpy: {:s}'.format(np.__version__))
import sklearn
from sklearn.ensemble import IsolationForest
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
print('Scikit Learn Version:', sklearn.__version__)

zat: 0.4.7
Pandas: 2.2.2
Numpy: 1.26.2
Scikit Learn Version: 1.3.2


In [28]:
labels = np.array(df_dumie['Malicious'])
features = df_dumie.drop(['Traffic_Labeled','Malicious'], axis = 1)
features = features.select_dtypes(exclude=['object'])
feature_list = list(features.columns)

In [29]:
train_features, test_features, train_labels, test_labels = train_test_split(features, labels, test_size = 0.25, random_state = 42)

feature scaling

In [30]:
sc_X = StandardScaler()
train_features = sc_X.fit_transform(train_features)
test_features = sc_X.transform(test_features)

In [31]:
clf = RandomForestClassifier(n_estimators = 200, max_depth=20,) 

In [32]:
train_features[0]

array([ 0.        ,  1.49132127,  1.09832459, -0.06735393, -0.06479325,
       -0.09615585,  0.36849844, -0.35227064,  0.0676726 , -0.0256503 ,
       -0.02690315, -0.05646304])

Training the model on the training dataset

In [33]:
clf.fit(train_features, train_labels)
 
# performing predictions on the test dataset
y_pred = clf.predict(test_features)
 
# metrics are used to find accuracy or error
from sklearn import metrics 
print()
 
# using metrics module for accuracy calculation
print("ACCURACY OF THE MODEL: ", metrics.accuracy_score(test_labels, y_pred))


ACCURACY OF THE MODEL:  0.6342297323952922


In [43]:
import pickle
pickle.dump(clf, open('model.pkl', 'wb'))